In [1]:
import FPCAnalysis
import numpy as np
import os

In [2]:
#load the input parameters
path = '/home/crbrown/Simulations/Tristan/htg_perp_bigrun/'
num = '700'

#Read the inputs
inputpath = path+'/input'
path = path+'/output/'
inputs = FPCAnalysis.dtr.load_input(inputpath)

#read the parameters of the simulation
params = FPCAnalysis.dtr.load_params(path,num)

#load the fields data
dfields = FPCAnalysis.dtr.load_fields(path,num,normalizeFields=True)

lowerresolution = False #lower field resolution for speed!
if(lowerresolution):
    print('dfields["ex"].shape before:',dfields['ex'].shape)
    dfields = FPCAnalysis.ao.avg_dict(dfields,binidxsz=[1,8,8]) #note, each element in binidxsz must be an integer divisor of the shape of its respective axis
    print('dfields["ex"].shape after:',dfields['ex'].shape)

#load particles, with velocities normalzied to upstream thermal species velocity
dpar_elec, dpar_ion = FPCAnalysis.dtr.load_particles(path,num,normalizeVelocity=True)

#Get needed parameters for boosting (needed for normalizations conversions)
beta0 = FPCAnalysis.anl.compute_beta0_tristanmp1(params,inputs)
betai,betae= FPCAnalysis.anl.get_betai_betae_from_tot_and_ratio(beta0,inputs['temperature_ratio'])
dt = params['c']/params['comp'] #in units of wpe
dt,c = FPCAnalysis.anl.norm_constants_tristanmp1(params,dt,inputs) #in units of wci (\omega_ci) and va respectively- as required by the rest of the scripts

print("Remember to enter shock velocity in this block!! (or 0 for simulation frame)")
vshock = 1.5625375379202415 #there exists ad hoc functions for tracking the shock in the library (for example, see notebook 4_2), but they might not always work, especially for qpar shocks or reforming shocks

#Boost to shock rest frame
dfields = FPCAnalysis.ft.lorentz_transform_vx_c(dfields,vshock,c) #note: we only boost one frame
dpar_ion = FPCAnalysis.ft.shift_particles_tristan(dpar_ion, vshock, betai, betae, params['mi']/params['me'], isIon=True)
dpar_elec = FPCAnalysis.ft.shift_particles_tristan(dpar_elec, vshock, betai, betae, params['mi']/params['me'], isIon=False)

dfluc = FPCAnalysis.anl.remove_average_fields_over_yz(dfields)
dfavg = FPCAnalysis.anl.get_average_fields_over_yz(dfields)


Remember to enter shock velocity in this block!! (or 0 for simulation frame)


In [4]:
flnmprefix = 'fftfilterbelow'
filterabove = False
kycutoff = 15.
dfluchighpass = FPCAnalysis.anl.yz_fft_filter_range(dfluc,kycutoff,filterabove,dontfilter=False,verbose=False)

flnmprefix = 'fftfilterabove'
filterabove = True
kycutoff = 15.
dfluclowpass = FPCAnalysis.anl.yz_fft_filter_range(dfields,kycutoff,filterabove,dontfilter=False,verbose=False,keys=['ex','ey','ez','bx','by','bz'])


In [6]:
vmaxion = 20.
dvion = .5
vmaxelec = 15.
dvelec = .5

vrmaxion = 20.
vrmaxelec = 20.
nrbins = 10

x1 = 8.4
x2 = 8.5
y1 = -10
y2 = 10
z1 = -10
z2 = 10

In [7]:
#Create arrays of different inputs for the compute_hist_and_cor call
#This will set things up to compute the many different forms/separations of the FPC

dfieldsinput = [dfields,dfluc,dfavg,dfields,dfluc,dfavg,dfields,dfields,dfields,dfields,dfields,dfields,dfields] #These are references, so not much more memory should be used
dir1inputs = ['etot','etot','etot','ex','ex','ex','epar','epar','epar','epar','epar','epar','epar','epar']
dir2inputs = ['etot','etot','etot','ey','ey','ey','eperp1','eperp1','eperp1','eperp1','eperp1','eperp1','eperp1']
dir3inputs = ['etot','etot','etot','ez','ez','ez','eperp2','eperp2','eperp2','eperp2','eperp2','eperp2','eperp2']
useBoxFACinputs = [True,True,True,True,True,True,True,True,True,False,False,True,True]
altcorfieldsinput = [None,None,None,None,None,None,None,dfluc,dfavg,None,dfluc,dfluchighpass,dfluclowpass]
flnmtags = ['totetot','flucetot','avgetot','tot','fluc','avg','fac','facfluc','facavg','faclocal','facfluclocal','fachighpassfluc','faclowpassfluc']

#make output folders
directory_path = os.path.join("figures")
os.makedirs(directory_path, exist_ok=True)
directory_path = os.path.join("figures", "ion")
os.makedirs(directory_path, exist_ok=True)
directory_path = os.path.join("figures", "elec")
os.makedirs(directory_path, exist_ok=True)

In [8]:
x1s = np.arange(7.5,9,.1)
x1s = np.asarray([float(round(xval,1)) for xval in x1s])
x2s = x1s+.1
x2s = np.asarray([float(round(xval,1)) for xval in x2s])
print(x1s,x2s)         


[7.5 7.6 7.7 7.8 7.9 8.  8.1 8.2 8.3 8.4 8.5 8.6 8.7 8.8 8.9] [7.6 7.7 7.8 7.9 8.  8.1 8.2 8.3 8.4 8.5 8.6 8.7 8.8 8.9 9. ]


In [9]:
for _index in range(0,len(x1s)):
    x1 = x1s[_index]
    x2 = x2s[_index]
    
    #compute for ions
    for _i in range(0,len(flnmtags)):
        if(not(flnmtags[_i] in ['totetot','flucetot','avgetot'])):
            vxi, vyi, vzi, totalPtcl, hist, corex = FPCAnalysis.fpc.compute_hist_and_cor(vmaxion, dvion, x1, x2, y1, y2, z1, z2, dpar_ion, dfieldsinput[_i], dir1inputs[_i], altcorfields=altcorfieldsinput[_i], useBoxFAC=useBoxFACinputs[_i])
            vxi, vyi, vzi, totalPtcl, hist, corey = FPCAnalysis.fpc.compute_hist_and_cor(vmaxion, dvion, x1, x2, y1, y2, z1, z2, dpar_ion, dfieldsinput[_i], dir2inputs[_i], altcorfields=altcorfieldsinput[_i], useBoxFAC=useBoxFACinputs[_i])
            vxi, vyi, vzi, totalPtcl, hist, corez = FPCAnalysis.fpc.compute_hist_and_cor(vmaxion, dvion, x1, x2, y1, y2, z1, z2, dpar_ion, dfieldsinput[_i], dir3inputs[_i], altcorfields=altcorfieldsinput[_i], useBoxFAC=useBoxFACinputs[_i])
        else:
            vxi, vyi, vzi, totalPtcl, hist, core = FPCAnalysis.fpc.compute_hist_and_cor(vmaxion, dvion, x1, x2, y1, y2, z1, z2, dpar_ion, dfieldsinput[_i], dir1inputs[_i], altcorfields=altcorfieldsinput[_i], useBoxFAC=useBoxFACinputs[_i])
        directory_path = os.path.join("figures", "ion", flnmtags[_i])
        os.makedirs(directory_path, exist_ok=True)
        flnm = 'figures/ion/'+flnmtags[_i]+'/ion'+flnmtags[_i]+'x1-'+str(x1)+'_'+'x2-'+str(x2)+'_'+'y1-'+str(y1)+'_'+'y2-'+str(y2)+'_'+'z1-'+str(z1)+'_'+'z2-'+str(z2)
        
        if (flnmtags[_i] in ['fac','facfluc','facavg','faclocal','facfluclocal','totetot','flucetot','avgetot','fachighpassfluc','faclowpassfluc']):
            plotFAC = True
        else:
            plotFAC = False
    
        if (flnmtags[_i] in ['avg','facavg','avgetot']):
            plotAvg = True
        else:
            plotAvg = False
    
        if (flnmtags[_i] in ['fluc','facfluc','facfluclocal','flucetot','fachighpassfluc','faclowpassfluc']):
            plotFluc = True
        else:
            plotFluc = False

        if (flnmtags[_i] in ['faclowpassfluc']):
            isLowPass=True
        else:
            isLowPass=False
        
        if (flnmtags[_i] in ['fachighpassfluc']):
            isHighPass=True
        else:
            isHighPass=False
    
        if(not(flnmtags[_i] in ['totetot','flucetot','avgetot'])):
            FPCAnalysis.pltvv.project_and_plot_supergrid(vxi,vyi,vzi,vmaxion,hist,corex,corey,corez,flnm,plotFAC=plotFAC,plotAvg=plotAvg,plotFluc=plotFluc,isIon=True)
            if(flnmtags[_i] in ['fac','facfluc','facavg','faclocal','facfluclocal','totetot','flucetot','avgetot']):
                flnm = 'figures/ion/'+flnmtags[_i]+'/iongyro'+flnmtags[_i]+str(x1)+'_'+'x2-'+str(x2)+'_'+'y1-'+str(y1)+'_'+'y2-'+str(y2)+'_'+'z1-'+str(z1)+'_'+'z2-'+str(z2)
                vpargyro,vperpgyro,corepargyro,coreperpgyro  = FPCAnalysis.anl.compute_gyro_fpc_from_cart_fpc(vxi,vyi,vzi,corez,corey,corex,vrmaxion,nrbins)
                FPCAnalysis.pltvv.plot_gyro(vpargyro,vperpgyro,corepargyro,coreperpgyro,flnm=flnm,isIon=True,plotLog=False,plotAvg = plotAvg, plotFluc = plotFluc)
                flnm = 'figures/ion/'+flnmtags[_i]+'/ioncompgyro'+'x1-'+flnmtags[_i]+str(x1)+'_'+'x2-'+str(x2)+'_'+'y1-'+str(y1)+'_'+'y2-'+str(y2)+'_'+'z1-'+str(z1)+'_'+'z2-'+str(z2)
                vpargyro,vperpgyro,corepargyro,coreperp1gyro,coreperp2gyro  = FPCAnalysis.anl.compute_compgyro_fpc_from_cart_fpc(vxi,vyi,vzi,corez,corey,corex,vrmaxion,nrbins) #Note: corez, corey, corex is the correct (albeit odd) ordering here...
                FPCAnalysis.pltvv.plot_gyro_3comp(vpargyro,vperpgyro,corepargyro,coreperp1gyro,coreperp2gyro,flnm=flnm,isIon=True,plotAvg = plotAvg, plotFluc = plotFluc)
            
        else:
            FPCAnalysis.pltvv.project_and_plot_supergrid_row(vxi,vyi,vzi,vmaxion,core,'Ctot',flnm,plotFAC=plotFAC,plotAvg=plotAvg,plotFluc=plotFluc,isIon=True)
            vpargyro,vperpgyro,cortotgyro = FPCAnalysis.anl.compute_gyro_fpc_from_cart_fpc_single(vxi,vyi,vzi,core,vrmaxion,nrbins)
            flnm = 'figures/ion/'+flnmtags[_i]+'/iongyro'+'x1-'+flnmtags[_i]+str(x1)+'_'+'x2-'+str(x2)+'_'+'y1-'+str(y1)+'_'+'y2-'+str(y2)+'_'+'z1-'+str(z1)+'_'+'z2-'+str(z2)
            FPCAnalysis.pltvv.plot_gyro_single(vpargyro,vperpgyro,cortotgyro,flnm=flnm,coresubscript='tot',isIon=True,plotAvg = plotAvg, plotFluc = plotFluc)
        

    #compute for elecs
    for _i in range(0,len(flnmtags)):
        if(not(flnmtags[_i] in ['totetot','flucetot','avgetot'])):
            vxe, vye, vze, totalPtcl, hist, corex = FPCAnalysis.fpc.compute_hist_and_cor(vmaxelec, dvelec, x1, x2, y1, y2, z1, z2, dpar_elec, dfieldsinput[_i], dir1inputs[_i], altcorfields=altcorfieldsinput[_i], useBoxFAC=useBoxFACinputs[_i])
            vxe, vye, vze, totalPtcl, hist, corey = FPCAnalysis.fpc.compute_hist_and_cor(vmaxelec, dvelec, x1, x2, y1, y2, z1, z2, dpar_elec, dfieldsinput[_i], dir2inputs[_i], altcorfields=altcorfieldsinput[_i], useBoxFAC=useBoxFACinputs[_i])
            vxe, vye, vze, totalPtcl, hist, corez = FPCAnalysis.fpc.compute_hist_and_cor(vmaxelec, dvelec, x1, x2, y1, y2, z1, z2, dpar_elec, dfieldsinput[_i], dir3inputs[_i], altcorfields=altcorfieldsinput[_i], useBoxFAC=useBoxFACinputs[_i])
        else:
            vxe, vye, vze, totalPtcl, hist, core = FPCAnalysis.fpc.compute_hist_and_cor(vmaxelec, dvelec, x1, x2, y1, y2, z1, z2, dpar_elec, dfieldsinput[_i], dir1inputs[_i], altcorfields=altcorfieldsinput[_i], useBoxFAC=useBoxFACinputs[_i])
        directory_path = os.path.join("figures", "elec", flnmtags[_i])
        os.makedirs(directory_path, exist_ok=True)
        flnm = 'figures/elec/'+flnmtags[_i]+'/elec'+flnmtags[_i]+'x1-'+str(x1)+'_'+'x2-'+str(x2)+'_'+'y1-'+str(y1)+'_'+'y2-'+str(y2)+'_'+'z1-'+str(z1)+'_'+'z2-'+str(z2)
        
        if (flnmtags[_i] in ['fac','facfluc','facavg','faclocal','facfluclocal','totetot','flucetot','avgetot','fachighpassfluc','faclowpassfluc']):
            plotFAC = True
        else:
            plotFAC = False
    
        if (flnmtags[_i] in ['avg','facavg','avgetot']):
            plotAvg = True
        else:
            plotAvg = False
    
        if (flnmtags[_i] in ['fluc','facfluc','facfluclocal','flucetot','fachighpassfluc','faclowpassfluc']):
            plotFluc = True
        else:
            plotFluc = False

        if (flnmtags[_i] in ['faclowpassfluc']):
            isLowPass=True
        else:
            isLowPass=False
        
        if (flnmtags[_i] in ['fachighpassfluc']):
            isHighPass=True
        else:
            isHighPass=False
    
    
        if(not(flnmtags[_i] in ['totetot','flucetot','avgetot'])):
            FPCAnalysis.pltvv.project_and_plot_supergrid(vxe,vye,vze,vmaxelec,hist,corex,corey,corez,flnm,plotFAC=plotFAC,plotAvg=plotAvg,plotFluc=plotFluc,isIon=False)
            if(flnmtags[_i] in ['fac','facfluc','facavg','faclocal','facfluclocal','totetot','flucetot','avgetot']):
                flnm = 'figures/elec/'+flnmtags[_i]+'/elecgyro'+flnmtags[_i]+str(x1)+'_'+'x2-'+str(x2)+'_'+'y1-'+str(y1)+'_'+'y2-'+str(y2)+'_'+'z1-'+str(z1)+'_'+'z2-'+str(z2)
                vpargyro,vperpgyro,corepargyro,coreperpgyro  = FPCAnalysis.anl.compute_gyro_fpc_from_cart_fpc(vxe,vye,vze,corez,corey,corex,vrmaxelec,nrbins)
                FPCAnalysis.pltvv.plot_gyro(vpargyro,vperpgyro,corepargyro,coreperpgyro,flnm=flnm,isIon=False,plotLog=False,plotAvg = plotAvg, plotFluc = plotFluc)
                flnm = 'figures/elec/'+flnmtags[_i]+'/eleccompgyro'+'x1-'+flnmtags[_i]+str(x1)+'_'+'x2-'+str(x2)+'_'+'y1-'+str(y1)+'_'+'y2-'+str(y2)+'_'+'z1-'+str(z1)+'_'+'z2-'+str(z2)
                vpargyro,vperpgyro,corepargyro,coreperp1gyro,coreperp2gyro  = FPCAnalysis.anl.compute_compgyro_fpc_from_cart_fpc(vxe,vye,vze,corez,corey,corex,vrmaxelec,nrbins) #Note: corez, corey, corex is the correct (albeit odd) ordering here...
                FPCAnalysis.pltvv.plot_gyro_3comp(vpargyro,vperpgyro,corepargyro,coreperp1gyro,coreperp2gyro,flnm=flnm,isIon=False,plotAvg = plotAvg, plotFluc = plotFluc)
            
        else:
            FPCAnalysis.pltvv.project_and_plot_supergrid_row(vxe,vye,vze,vmaxelec,core,'Ctot',flnm,plotFAC=plotFAC,plotAvg=plotAvg,plotFluc=plotFluc,isIon=False)
            vpargyro,vperpgyro,cortotgyro = FPCAnalysis.anl.compute_gyro_fpc_from_cart_fpc_single(vxe,vye,vze,core,vrmaxelec,nrbins)
            flnm = 'figures/elec/'+flnmtags[_i]+'/elecgyro'+'x1-'+flnmtags[_i]+str(x1)+'_'+'x2-'+str(x2)+'_'+'y1-'+str(y1)+'_'+'y2-'+str(y2)+'_'+'z1-'+str(z1)+'_'+'z2-'+str(z2)
            FPCAnalysis.pltvv.plot_gyro_single(vpargyro,vperpgyro,cortotgyro,flnm=flnm,coresubscript='tot',isIon=False,plotAvg = plotAvg, plotFluc = plotFluc)
        
        